## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

children — количество детей в семье<br>
days_employed — общий трудовой стаж в днях<br>
dob_years — возраст клиента в годах<br>
education — уровень образования клиента<br>
education_id — идентификатор уровня образования<br>
family_status — семейное положение
family_status_id — идентификатор семейного положения
gender — пол клиента
income_type — тип занятости
debt — имел ли задолженность по возврату кредитов
total_income — ежемесячный доход
purpose — цель получения кредита

In [1]:
import pandas as pd

In [2]:

borrowers=pd.read_csv('/datasets/data.csv')
borrowers.head(10)

children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   
5  гражданский брак                 1      M   компаньон     0  255763.565419   
6   женат / замужем                 0      F   компаньон     0  240525.971920   
7   женат / замужем                 0      M   сотрудник     0  135823.934197   
8  гражданский брак                 1      F   сотрудник     0   95856.832424   
9   женат / замужем                 0      M   сотрудник     0  144425.938277   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жилья  
3  дополнительное образование  
4             сыграть свадьбу  
5               покупка жилья  
6           операции с жильем  
7                 образование  
8       на проведение свадьбы  
9     покупка жилья для семьи

In [3]:
# Исследуем дата фрейм
borrowers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В дата фрейме 21525 строк, но в столбцах total_income и days_employed их меньше, что говорит о том, что в них есть пропуски. 

### Вывод

Собрали данные для дальнейшего анализа, выявили, что есть пропуски в столбце total_income и days_employed. Скорее всего они стали случайными, но это мы проверяем в следующих главах. 

### Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
# Проверим, действительно ли пропуски данных встречаются только в столбце total_income
borrowers.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
# Действительно, больше нигде не встречаются пропуски в данных. Проверим теперь, с чем это может быть связано
borrowers[borrowers.total_income.isnull()].head(10)

children  days_employed  dob_years education  education_id  \
12         0            NaN         65   среднее             1   
26         0            NaN         41   среднее             1   
29         0            NaN         63   среднее             1   
41         0            NaN         50   среднее             1   
55         0            NaN         54   среднее             1   
65         0            NaN         21   среднее             1   
67         0            NaN         52    высшее             0   
72         1            NaN         32    высшее             0   
82         2            NaN         50    высшее             0   
83         0            NaN         52   среднее             1   

            family_status  family_status_id gender  income_type  debt  \
12       гражданский брак                 1      M    пенсионер     0   
26        женат / замужем                 0      M  госслужащий     0   
29  Не женат / не замужем                 4      F    пенсионер     0   
41        женат / замужем                 0      F  госслужащий     0   
55       гражданский брак                 1      F    пенсионер     1   
65  Не женат / не замужем                 4      M    компаньон     0   
67        женат / замужем                 0      F    пенсионер     0   
72        женат / замужем                 0      M  госслужащий     0   
82        женат / замужем                 0      F    сотрудник     0   
83        женат / замужем                 0      M    сотрудник     0   

    total_income                                purpose  
12           NaN                        сыграть свадьбу  
26           NaN                            образование  
29           NaN       строительство жилой недвижимости  
41           NaN       сделка с подержанным автомобилем  
55           NaN                        сыграть свадьбу  
65           NaN  операции с коммерческой недвижимостью  
67           NaN                покупка жилья для семьи  
72           NaN  операции с коммерческой недвижимостью  
82           NaN                                  жилье  
83           NaN                                  жилье

In [6]:
# При взгляде на первые 10 строк датафрейма можно заметить, что пропуски здесь случайны. Предлагаю заменить на медианные значения, но 
# с учетом занятости человека. Более 50% являются сотрудниками
borrowers.income_type.value_counts(normalize=True)

сотрудник          0.516562
компаньон          0.236237
пенсионер          0.179141
госслужащий        0.067782
безработный        0.000093
предприниматель    0.000093
в декрете          0.000046
студент            0.000046
Name: income_type, dtype: float64

In [7]:
# Посмотрим теперь средние доходы по всем занятостям
borrowers.groupby(['income_type']).agg({'total_income':['mean','count','median']})

total_income                      
                          mean  count         median
income_type                                         
безработный      131339.751676      2  131339.751676
в декрете         53829.130729      1   53829.130729
госслужащий      170898.309923   1312  150447.935283
компаньон        202417.461462   4577  172357.950966
пенсионер        137127.465690   3443  118514.486412
предприниматель  499163.144947      1  499163.144947
сотрудник        161380.260488  10014  142594.396847
студент           98201.625314      1   98201.625314

In [8]:
borrowers[borrowers.income_type=='предприниматель']

children  days_employed  dob_years education  education_id  \
5936          0            NaN         58    высшее             0   
18697         0    -520.848083         27    высшее             0   

          family_status  family_status_id gender      income_type  debt  \
5936    женат / замужем                 0      M  предприниматель     0   
18697  гражданский брак                 1      F  предприниматель     0   

        total_income                     purpose  
5936             NaN  покупка жилой недвижимости  
18697  499163.144947       на проведение свадьбы

In [9]:
# Пропуски примерно в 10% случаев (Исключение - предприниматель, в данных только две строчки с таким значением, и одна - с пропуском)
borrowers[borrowers.total_income.isnull()].groupby(['income_type']).agg({'income_type':['count']})

income_type
                      count
income_type                
госслужащий             147
компаньон               508
пенсионер               413
предприниматель           1
сотрудник              1105

In [10]:
# Заменим пропуски (уверен, что этот код работает долго, так как он обращается к каждой ячейке с пропущенным значением, но уменьшает объем строчек) 
for i in borrowers[borrowers.total_income.isnull()].groupby(['income_type']).agg({'income_type':['count']}).index:
    for m in borrowers[borrowers.total_income.isnull()].loc[borrowers[borrowers.total_income.isnull()].income_type==i,'total_income'].index:
        borrowers.at[m,'total_income'] = borrowers.groupby(['income_type']).agg({'total_income':['median']})['total_income'].loc[i,'median']
        
        
        

In [15]:
# Проверим теперь
borrowers[borrowers.total_income.isnull()].loc[borrowers[borrowers.total_income.isnull()].income_type==i,'total_income']

Series([], Name: total_income, dtype: float64)

In [18]:
# Посмотрим, осталичьс ли у нас пропуски теперь.

borrowers.isnull().sum()

# Все чисто. Осталось только days_employed с пропусками, но для дальнейшего анализа этот столбец нам не требуется, поэтому
# оставлю его без изменений

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

In [19]:
# Посмотрим, как поменялись значения после обработки.
borrowers.groupby(['income_type']).agg({'total_income':['mean','count','median']})

# Медианные значения не поменялись, средние значения изменились незначительно. 

total_income                      
                          mean  count         median
income_type                                         
безработный      131339.751676      2  131339.751676
в декрете         53829.130729      1   53829.130729
госслужащий      168837.854082   1459  150447.935283
компаньон        199414.466116   5085  172357.950966
пенсионер        135133.907484   3856  118514.486412
предприниматель  499163.144947      2  499163.144947
сотрудник        159513.331868  11119  142594.396847
студент           98201.625314      1   98201.625314

### Вывод

Провели анализ пропущенных значений. Скорее всего пропуски были случайными. Заменили их медианными значения по типу занятости. 

### Замена типа данных

In [21]:
borrowers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [22]:
# Заменим данные по размеру дохода на целые числа
borrowers['total_income'] = borrowers['total_income'].astype(int)

In [23]:
# Сделаем проверку
borrowers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Вывод

Заменили данные по размеру дохода на вещественные значения. В виде без большого количества чисел после запятой доходы выглядят более реалистично. 

### Обработка дубликатов

In [29]:
# Есть около 54 дубликатов, которые стоит удалить
borrowers.duplicated().sum()

54

In [31]:
borrowers=borrowers.drop_duplicates()

In [32]:
# Новая проверка на поиск дубликатов
borrowers.duplicated().sum()

0

In [33]:
# Можно увидеть, что в некоторых стобцах присутствует наличие разного регистра в данных.
print(borrowers['education'].value_counts())
print()
print(borrowers['family_status'].value_counts())

# Для столбца education это значимая проблема, так как одно и то же значения записано по-разному
# Для столбца family_status разный регистр в написании не создает новое уникальное значение. Поэтому приводить к одному регистру не имеет смысла


среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64


In [38]:
borrowers.loc[:,'education'] = borrowers['education'].str.lower()

In [12]:
# В столбце education все приведем к одному регистру. 
borrowers['education'] = borrowers['education'].str.lower()

In [13]:
# Код ревьюера
pd.set_option('mode.chained_assignment', None)

In [39]:
# Сделаем проверку
borrowers.education.value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

### Вывод

В двух столбцах (education и family_status) было обнаружено разный регистр в написании данных. Однако в одном из них (education) это создавало большую проблему для дальнейшего исследования данных (создавало больше уникальных значений). В ходе анализа все значения были приведены к нижнему регистру, что сократило сильно количество уникальных значений. 

### Лемматизация

In [40]:
# В данных представлено более 30 уникальных целей, под которые выдавался кредит. 
# Однако еслм присмотреться, то из них можно выделить несколько групп, которые повторяются в целях - это автомобиль, образование, свадьба, строительство/покупка недвижимости, ремонт
borrowers.purpose.value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [41]:
# Но цели встречаются не в начальной форме, поэтому требуется лемматизация. 
from collections import Counter
from pymystem3 import Mystem
m = Mystem()
borrowers.loc[:,'purpose']=borrowers['purpose'].apply(m.lemmatize)

In [42]:
# Напишем функцию, которая бы приписывала более короткую цель.
def purpose(row):
    for purpose in row:
        if 'свадьба' in row:
            return 'свадьба'
        if 'образование' in row:
            return 'образование'
#         if 'коммерческий' in row:
#             return 'коммерческая недвижимость'
        if 'жилье' in row:
            return 'недвижимость'
        if 'недвижимость' in row:
            return 'недвижимость'
        if 'автомобиль' in row:
            return 'автомобиль'
        if 'машина' in row:
            return 'автомобиль'


# В итоге отказался от вариантов коммерческая недвижимость и ремонт, поскольку они могут быть не полностью заполнены. 
# Цель недвижимость может включать в себя операции с жилой и некоммерческой недвижимостью, а также ремонт в ней. 

In [44]:
borrowers.loc[:,'purePurpose']=borrowers.loc[:,'purpose'].apply(purpose)
borrowers['purePurpose'].value_counts()

недвижимость    10814
автомобиль       4308
образование      4014
свадьба          2335
Name: purePurpose, dtype: int64

### Вывод

Провели анализ целей, выявили, что многие цели похожи друг на друга и для того, чтобы сгруппировать их, применили лемматизацию, чтобы привести к изначальной форме. В итоге выявили 4 цели: недвижимость, автомобиль, образование и свадьба.
Выбор данных лемм объясняется тем, что все они встречаются в каждой цели по одному разу (жилье и недвижимость - синонимы), при этом являются разными как по названию кредитов, так и по условиям кредитования, по процентным ставкам, длительности кредитов и социально-демографического портрета заемщиков. Например, молодые люди чаще возьмут кредит на образование и свадьбу, чем ипотеку. 

### Категоризация данных

In [45]:
borrowers.total_income.nunique()
# В категоризации нуждается графа total_income, поскольку в ней более 18 тыс уникальных значений. 

18608

In [48]:
# Предлагаю выявить 4 равные группы:
def income(row):
    if row <= borrowers[borrowers.total_income<borrowers.total_income.median()].total_income.median():
        return 1
    if row <= borrowers.total_income.median():
        return 2
    if row <= borrowers[borrowers.total_income>borrowers.total_income.median()].total_income.median():
        return 3
    if row > borrowers[borrowers.total_income>borrowers.total_income.median()].total_income.median():
        return 4
# Плохо, что функция работает медленно. Но ее плюс в том, что она может подходить для автоматизации будущем, поскольку 
# использует чисто формальные абстракции вроде медианного значения, а не конкретное число. 

In [49]:
borrowers.loc[:,'income_class']=borrowers.loc[:,'total_income'].apply(income)

In [50]:
# Проверим, что получилось. 
borrowers.income_class.value_counts(normalize=True)

# Получилось почти идеально. 

2    0.277910
3    0.247171
4    0.247124
1    0.227796
Name: income_class, dtype: float64

### Вывод

Провели категоризацию данных по доходу, выявили 4 группы. 

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [51]:
# В столбце children есть аномальные явления -1 ребенок и 20 детей, которые произошли случайно. 
# Возможно, это произошло по следующим причинам: -1 может подразумевать 1, просто минус появился в результате неверного заполнения 
# или ошибка в парсинге данных  
# 20 - аномальное явление, которое встречается очень редко. Если бы в данных было такое распределение: от 0 до 20 детей с шагом 1,
# то еще можно поверить. Но в данном случае 20 идет следом за 5, так что предполагаю, что это неверено записанное число 2 по тем же
# самым причинам, что и с -1. 
borrowers.children.value_counts()

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [52]:
# Предлагаю -1 поменять на 1, а 20 - на 2
borrowers.loc[borrowers['children'] == -1,'children'] = 1
borrowers.loc[borrowers['children'] == 20,'children'] = 2

In [53]:
borrowers.pivot_table('debt','children',aggfunc='count')

debt
children       
0         14107
1          4856
2          2128
3           330
4            41
5             9

In [55]:
# Посчитаем конверсию среди невернувших долг в каждой группе:
for i in borrowers.children.unique():
    print('Процент невернувших долг у людей, имеющих ',i,'детей составляет {:.2%}'.format(len(borrowers[borrowers.children==i][borrowers[borrowers.children==i].debt==1])/len(borrowers[borrowers.children==i])))

Процент невернувших долг у людей, имеющих  1 детей составляет 9.16%
Процент невернувших долг у людей, имеющих  0 детей составляет 7.54%
Процент невернувших долг у людей, имеющих  3 детей составляет 8.18%
Процент невернувших долг у людей, имеющих  2 детей составляет 9.49%
Процент невернувших долг у людей, имеющих  4 детей составляет 9.76%
Процент невернувших долг у людей, имеющих  5 детей составляет 0.00%


In [57]:
# Конверсия у группы без детей значительно ниже (на 1-2 процентных пункта). 
# Также задолженностей нет у группы с 5 детьми, однако эта группа очень маленькая, и возможно результат нельзя назвать значимыи для данной группы

#  Для закрепления данного инсайта предлагаю провести категоризацию и проверить отсутствие отсутствие/наличие детей на возврат кредита 

def children(row):
    if row==0:
        return 0
    if row!=0:
        return 1

borrowers.loc[:,'childrenBool']=borrowers.loc[:,'children'].apply(children)    



In [58]:
# Конверсия разная, что говорит, что наличие детей отрицательно влияет на возвращение долга. 
borrowers.pivot_table('debt','childrenBool',aggfunc='mean')

debt
childrenBool          
0             0.075353
1             0.092070

### Вывод

Провели анализ и выявили, что наличие детей отрицательно может повлиять на возврат кредита. Так,процент невозврата среди бездетных людей составляет 7.5, а у тех, кто имеет детей - 9.1%. Более того, существует зависимость между количеством детей. Например, вероятность невозврата кредита с одним ребенком составляет 9.2%, с 2 - 9.5%, у кого 3 ребенка - 8.2%. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [59]:
# В большинстве своем в данных представлены женатые / замужние граждане, что говорит о том, что выборка для данной категории будет 
# более полной
borrowers.family_status.value_counts(normalize=True)

женат / замужем          0.574915
гражданский брак         0.193889
Не женат / не замужем    0.130874
в разводе                0.055656
вдовец / вдова           0.044665
Name: family_status, dtype: float64

In [60]:
borrowers.pivot_table('debt','family_status',aggfunc='mean')

debt
family_status                  
Не женат / не замужем  0.097509
в разводе              0.071130
вдовец / вдова         0.065693
гражданский брак       0.093202
женат / замужем        0.075421

In [61]:
borrowers.pivot_table('dob_years','family_status',aggfunc='mean')

dob_years
family_status                   
Не женат / не замужем  38.363701
в разводе              45.517992
вдовец / вдова         56.503650
гражданский брак       42.107615
женат / замужем        43.548931

In [62]:
borrowers.pivot_table('dob_years','family_status',aggfunc='median')

dob_years
family_status                   
Не женат / не замужем         36
в разводе                     46
вдовец / вдова                58
гражданский брак              41
женат / замужем               43

### Вывод

Наибольший процент невозврата среди тех, кто не женат / не замужем и находится в гражданском браке. Возможно, такое происходит, поскольку в данную категорию обычно попадают более молодые люди (Таблица выше это показывает), у которых чаще всего бывает нестабильность в доходах. 
В то же время вдовцы с более высоким средним возрастом показали самую маленькую конверсию в задолженность, что говорит о том, что данная категория более ответственно относится к займу денег.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [81]:
# Количество людей в каждой группе таково, что превалируют в выборке люди из среднего класса. 
borrowers.income_class.value_counts()

2    5967
3    5307
4    5306
1    4891
Name: income_class, dtype: int64

In [82]:
borrowers[borrowers['debt']==1].income_class.value_counts(normalize=True)

2    0.301551
3    0.260770
1    0.221137
4    0.216542
Name: income_class, dtype: float64

In [80]:
classes=pd.concat([borrowers.income_class.value_counts(),
           borrowers[borrowers['debt']==1].income_class.value_counts(),
          (borrowers.pivot_table('debt','income_class',aggfunc='mean')*100).round(1)],axis=1)
classes.columns=['Количество людей в каждой категории','Количество задолжников','Доля задолжников, %']
classes
# Но и как показал анализ, что люди со средним достатком (2 и 3) чаще всего не возврвщают кредиты
# Эти результаты валидны, поскольку группы, изначально выбранные для анализа были сформированы почти одинаково, чтобы избежать 
# случайных ошибок, повлияющих на результат. 

Количество людей в каждой категории  Количество задолжников  \
1                                 4891                     385   
2                                 5967                     525   
3                                 5307                     454   
4                                 5306                     377   

   Доля задолжников, %  
1                  7.9  
2                  8.8  
3                  8.6  
4                  7.1

### Вывод

Граждане из среднего класса менее ответственно подходят к займу денег, поскольку показали более высокую конверсию в задолженность. Богатые - имеют меньше всего золженностей.

- Как разные цели кредита влияют на его возврат в срок?

In [38]:
data1.pivot_table('debt','purePurpose',aggfunc='mean')

debt
purePurpose           
автомобиль    0.093395
недвижимость  0.072140
образование   0.091994
свадьба       0.079216

### Вывод

Кредиты, взятые на автомобиль и образование, чаще не возвращаются, чем на другие цели. 

### Шаг 4. Общий вывод

Были проанализированы различные факторы, влияющие на возврат кредитов. 
Но для того чтобы дать на этот вопрос ответы, пришлось очень много делать преобработку данных, поскольку полученный дата фрейм был далек от идеала. 
Так, например, в столбце total_income были выявлены пропуски в данных, которые нельзя было оставить, но и удалять данные не хотелось, поэтому был применен метод замены пропущенных значений на медианное, исходя из того, к какой профессиональной группе человек принадлежал. 
Также в данном столбце числа были записаны некорректно, а именно имели несколько чисел после запятой, а этого быть не могло, так как дробюность денег является конечной, поэтому все значения были переведены в целочисленный тип. 
В-третьих, обнаружены и удалены дубликаты, которые могли бы привести к смещению результатов нашего анализа. 
В-четвертых, все строки в столбце education к нижнему регистру, поскольку создавали много уникальных наименований, исходя из разных регистров. Это тоже не позволило бы нам четко анализировать результаты. 
И в-пятых, была проведена лемматизация целей кредитования и выявлены основные 4 группы (автомобиль, свадьба, недвижимость и образование), которые объединили под собой все цели, которые были представлены в данном столбце. 

В ходе аналитики было выявлено, что количество детей отрицательно влияет на возврат кредита. Так, процент задолжников среди бездетных детей значительно ниже (7.5), чем у людей с детьми (9.1). Также количество детей может по-разному влиять на возврат кредитов. Например, среди людей с 1 ребенком вероятность невозврата составляет 9.2%, с 2 - 9.5%, у кого 3 ребенка - 8.2.

Также на возврат кредита влияет семейное положение человека. Наибольший процент невозврата среди тех, кто не женат / не замужем и находится в гражданском браке. Гипотеза, почему так происходит, связана с возврастом человека. Молодые люди чаще всего нестабильны в своих доходах. 

Граждане из среднего класса менее ответственно подходят к займу денег, поскольку показали более высокую конверсию в задолженность. Богатые - имеют меньше всего золженностей.

И последнее, на возврат влияет и цель, на которую человек берет кредит. Так, например, вероятность невозвраста кредита чаще всего у тех, кто берет его для покупки автомобиля, а также для образования. 
